# 데모

## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
import warnings

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 학습데이터 로드

[03-pandas-eda.ipynb](https://github.com/kaggler-tv/dku-kaggle-class/blob/master/notebook/03-pandas-eda.ipynb)에서 생성한 `feature.csv` 피처파일 사용

In [4]:
data_dir = Path('./data/')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
n_fold = 10
n_class = 3
seed = 42

In [5]:
algo_name = 'lgb_hyperopt_last1'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = data_dir / f'{feature_name}.csv'
p_val_file = data_dir / f'{model_name}.val.csv'
p_tst_file = data_dir / f'{model_name}.tst.csv'
sub_file = data_dir / f'{model_name}.csv'

In [6]:
df = pd.read_csv(feature_file, index_col=0)
print(df.shape)
df.head()

(400000, 25)


,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,class,d_dered_u,d_dered_g,d_dered_r,d_dered_i,d_dered_z,d_dered_zi,d_dered_zr,d_dered_zg,d_dered_zu,d_dered_ir,d_dered_ig,d_dered_iu,d_dered_ru,d_dered_gu,d_obs_det
id,,,,,,,,,,,,,,,,,,,,,,,,,
0,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,2.9444,1.1898,0.0,-0.1397,-0.0790,-0.0544,-0.0403,-0.0307,-0.7232,-2.0462,-3.3488,-6.2153,-1.3230,-2.6257,-5.4921,-4.1691,-2.8665,6.1132
1,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,0.6931,1.2533,1.0,-0.0857,-0.0574,-0.0410,-0.0322,-0.0343,-0.1016,-0.2767,-0.8698,-1.8317,-0.1751,-0.7683,-1.7302,-1.5550,-0.9619,1.4427
2,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,1.0986,1.0225,0.0,-0.1787,-0.1388,-0.0963,-0.0718,-0.0540,-0.0257,-0.1440,-0.3905,-1.3116,-0.1183,-0.3649,-1.2859,-1.1676,-0.9211,1.8205
3,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,1.6094,1.2054,0.0,-0.3070,-0.1941,-0.1339,-0.1003,-0.0795,-0.2395,-0.5321,-1.4395,-5.7985,-0.2926,-1.2000,-5.5590,-5.2664,-4.3590,1.8640
4,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,2.6391,1.1939,0.0,-0.6820,-0.2653,-0.1794,-0.1339,-0.1067,-0.3144,-0.9871,-2.5579,-5.8955,-0.6727,-2.2436,-5.5811,-4.9084,-3.3376,4.5471


In [7]:
y = df[target_col].values[:320000]
df.drop(target_col, axis=1, inplace=True)
trn = df.iloc[:320000].values
tst = df.iloc[320000:].values
feature_name = df.columns.tolist()
print(y.shape, trn.shape, tst.shape)

(320000,) (320000, 24) (80000, 24)


# 첫번째 Hyperparameter Tuning

In [8]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.3, random_state=seed)

In [11]:
params = {
    "objective": "multiclass",
    "n_estimators": 10000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [63, 127, 255]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1),
    "min_child_samples": hp.choice('min_child_samples', [10, 25, 100, 173]),
    "max_depth" : hp.choice("max_depth", [30, 40, 50])
}

In [12]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=100,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:19<00:00, 13.93s/trial, best loss: 0.15774049333518958]
{'objective': 'multiclass', 'n_estimators': 10000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.014231095380921514, 'max_depth': 30, 'min_child_samples': 173, 'num_leaves': 255, 'subsample': 0.7000000000000001}


# Stratified K-Fold Cross Validation

In [13]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

## 첫번째 LightGBM 모델 학습

In [14]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=100)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.965621
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.945879
[3]	valid_0's multi_logloss: 0.92627
[4]	valid_0's multi_logloss: 0.907909
[5]	valid_0's multi_logloss: 0.890794
[6]	valid_0's multi_logloss: 0.873794
[7]	valid_0's multi_logloss: 0.858204
[8]	valid_0's multi_logloss: 0.841715
[9]	valid_0's multi_logloss: 0.825343
[10]	valid_0's multi_logloss: 0.810058
[11]	valid_0's multi_logloss: 0.79611
[12]	valid_0's multi_logloss: 0.782502
[13]	valid_0's multi_logloss: 0.768849
[14]	valid_0's multi_logloss: 0.755395
[15]	valid_0's multi_logloss: 0.742803
[16]	valid_0's multi_logloss: 0.730372
[17]	valid_0's multi_logloss: 0.71805
[18]	valid_0's multi_logloss: 0.705292
[19]	valid_0's multi_logloss: 0.693253
[20]	valid_0's multi_logloss: 0.681532
[21]	valid_0's multi_logloss: 0.669928
[22]	valid_0's multi_logloss: 0.658499
[23]	valid_0's multi_logloss: 0.647504
[24]	valid_0's multi_logloss

[208]	valid_0's multi_logloss: 0.181397
[209]	valid_0's multi_logloss: 0.18108
[210]	valid_0's multi_logloss: 0.180702
[211]	valid_0's multi_logloss: 0.180463
[212]	valid_0's multi_logloss: 0.18024
[213]	valid_0's multi_logloss: 0.179952
[214]	valid_0's multi_logloss: 0.17965
[215]	valid_0's multi_logloss: 0.179303
[216]	valid_0's multi_logloss: 0.17892
[217]	valid_0's multi_logloss: 0.178558
[218]	valid_0's multi_logloss: 0.17837
[219]	valid_0's multi_logloss: 0.178001
[220]	valid_0's multi_logloss: 0.177665
[221]	valid_0's multi_logloss: 0.177471
[222]	valid_0's multi_logloss: 0.177169
[223]	valid_0's multi_logloss: 0.176888
[224]	valid_0's multi_logloss: 0.176664
[225]	valid_0's multi_logloss: 0.176375
[226]	valid_0's multi_logloss: 0.176083
[227]	valid_0's multi_logloss: 0.175856
[228]	valid_0's multi_logloss: 0.175618
[229]	valid_0's multi_logloss: 0.175378
[230]	valid_0's multi_logloss: 0.175062
[231]	valid_0's multi_logloss: 0.174811
[232]	valid_0's multi_logloss: 0.174638
[233]

[414]	valid_0's multi_logloss: 0.156428
[415]	valid_0's multi_logloss: 0.156394
[416]	valid_0's multi_logloss: 0.156362
[417]	valid_0's multi_logloss: 0.156314
[418]	valid_0's multi_logloss: 0.156271
[419]	valid_0's multi_logloss: 0.156255
[420]	valid_0's multi_logloss: 0.156219
[421]	valid_0's multi_logloss: 0.15618
[422]	valid_0's multi_logloss: 0.156139
[423]	valid_0's multi_logloss: 0.156105
[424]	valid_0's multi_logloss: 0.156067
[425]	valid_0's multi_logloss: 0.156027
[426]	valid_0's multi_logloss: 0.155989
[427]	valid_0's multi_logloss: 0.155976
[428]	valid_0's multi_logloss: 0.155941
[429]	valid_0's multi_logloss: 0.155893
[430]	valid_0's multi_logloss: 0.155883
[431]	valid_0's multi_logloss: 0.155842
[432]	valid_0's multi_logloss: 0.155815
[433]	valid_0's multi_logloss: 0.155777
[434]	valid_0's multi_logloss: 0.155759
[435]	valid_0's multi_logloss: 0.155728
[436]	valid_0's multi_logloss: 0.155695
[437]	valid_0's multi_logloss: 0.15567
[438]	valid_0's multi_logloss: 0.155634
[4

[620]	valid_0's multi_logloss: 0.153318
[621]	valid_0's multi_logloss: 0.153308
[622]	valid_0's multi_logloss: 0.153309
[623]	valid_0's multi_logloss: 0.153315
[624]	valid_0's multi_logloss: 0.153313
[625]	valid_0's multi_logloss: 0.153307
[626]	valid_0's multi_logloss: 0.153308
[627]	valid_0's multi_logloss: 0.153296
[628]	valid_0's multi_logloss: 0.153289
[629]	valid_0's multi_logloss: 0.153279
[630]	valid_0's multi_logloss: 0.153271
[631]	valid_0's multi_logloss: 0.153264
[632]	valid_0's multi_logloss: 0.153255
[633]	valid_0's multi_logloss: 0.153248
[634]	valid_0's multi_logloss: 0.153244
[635]	valid_0's multi_logloss: 0.153243
[636]	valid_0's multi_logloss: 0.153238
[637]	valid_0's multi_logloss: 0.153227
[638]	valid_0's multi_logloss: 0.153222
[639]	valid_0's multi_logloss: 0.153222
[640]	valid_0's multi_logloss: 0.153232
[641]	valid_0's multi_logloss: 0.153236
[642]	valid_0's multi_logloss: 0.153218
[643]	valid_0's multi_logloss: 0.153223
[644]	valid_0's multi_logloss: 0.153216


[826]	valid_0's multi_logloss: 0.153092
[827]	valid_0's multi_logloss: 0.153079
[828]	valid_0's multi_logloss: 0.153083
[829]	valid_0's multi_logloss: 0.153086
[830]	valid_0's multi_logloss: 0.153088
[831]	valid_0's multi_logloss: 0.153088
[832]	valid_0's multi_logloss: 0.153084
[833]	valid_0's multi_logloss: 0.153093
[834]	valid_0's multi_logloss: 0.153102
[835]	valid_0's multi_logloss: 0.153108
[836]	valid_0's multi_logloss: 0.153113
[837]	valid_0's multi_logloss: 0.153108
[838]	valid_0's multi_logloss: 0.153111
[839]	valid_0's multi_logloss: 0.153107
[840]	valid_0's multi_logloss: 0.153105
[841]	valid_0's multi_logloss: 0.15311
[842]	valid_0's multi_logloss: 0.153109
[843]	valid_0's multi_logloss: 0.153116
[844]	valid_0's multi_logloss: 0.153119
[845]	valid_0's multi_logloss: 0.153117
[846]	valid_0's multi_logloss: 0.153118
[847]	valid_0's multi_logloss: 0.153118
[848]	valid_0's multi_logloss: 0.153113
[849]	valid_0's multi_logloss: 0.153105
[850]	valid_0's multi_logloss: 0.15311
[8

...

[52]	valid_0's multi_logloss: 0.436534
[53]	valid_0's multi_logloss: 0.431124
[54]	valid_0's multi_logloss: 0.42605
[55]	valid_0's multi_logloss: 0.421356
[56]	valid_0's multi_logloss: 0.417021
[57]	valid_0's multi_logloss: 0.412376
[58]	valid_0's multi_logloss: 0.407965
[59]	valid_0's multi_logloss: 0.403787
[60]	valid_0's multi_logloss: 0.399565
[61]	valid_0's multi_logloss: 0.395056
[62]	valid_0's multi_logloss: 0.390994
[63]	valid_0's multi_logloss: 0.387249
[64]	valid_0's multi_logloss: 0.383106
[65]	valid_0's multi_logloss: 0.379209
[66]	valid_0's multi_logloss: 0.375794
[67]	valid_0's multi_logloss: 0.372411
[68]	valid_0's multi_logloss: 0.369097
[69]	valid_0's multi_logloss: 0.365129
[70]	valid_0's multi_logloss: 0.361824
[71]	valid_0's multi_logloss: 0.358305
[72]	valid_0's multi_logloss: 0.355044
[73]	valid_0's multi_logloss: 0.35185
[74]	valid_0's multi_logloss: 0.348635
[75]	valid_0's multi_logloss: 0.345605
[76]	valid_0's multi_logloss: 0.342132
[77]	valid_0's multi_loglos

[259]	valid_0's multi_logloss: 0.170553
[260]	valid_0's multi_logloss: 0.170395
[261]	valid_0's multi_logloss: 0.170294
[262]	valid_0's multi_logloss: 0.170117
[263]	valid_0's multi_logloss: 0.169968
[264]	valid_0's multi_logloss: 0.169819
[265]	valid_0's multi_logloss: 0.169602
[266]	valid_0's multi_logloss: 0.169472
[267]	valid_0's multi_logloss: 0.169279
[268]	valid_0's multi_logloss: 0.169179
[269]	valid_0's multi_logloss: 0.169073
[270]	valid_0's multi_logloss: 0.168869
[271]	valid_0's multi_logloss: 0.168723
[272]	valid_0's multi_logloss: 0.168606
[273]	valid_0's multi_logloss: 0.168432
[274]	valid_0's multi_logloss: 0.168311
[275]	valid_0's multi_logloss: 0.168144
[276]	valid_0's multi_logloss: 0.168027
[277]	valid_0's multi_logloss: 0.167883
[278]	valid_0's multi_logloss: 0.167765
[279]	valid_0's multi_logloss: 0.167596
[280]	valid_0's multi_logloss: 0.16751
[281]	valid_0's multi_logloss: 0.167371
[282]	valid_0's multi_logloss: 0.167237
[283]	valid_0's multi_logloss: 0.167056
[

[465]	valid_0's multi_logloss: 0.156845
[466]	valid_0's multi_logloss: 0.156822
[467]	valid_0's multi_logloss: 0.156786
[468]	valid_0's multi_logloss: 0.156754
[469]	valid_0's multi_logloss: 0.156732
[470]	valid_0's multi_logloss: 0.156707
[471]	valid_0's multi_logloss: 0.156686
[472]	valid_0's multi_logloss: 0.156681
[473]	valid_0's multi_logloss: 0.156662
[474]	valid_0's multi_logloss: 0.156647
[475]	valid_0's multi_logloss: 0.156626
[476]	valid_0's multi_logloss: 0.156606
[477]	valid_0's multi_logloss: 0.156586
[478]	valid_0's multi_logloss: 0.156576
[479]	valid_0's multi_logloss: 0.156552
[480]	valid_0's multi_logloss: 0.156537
[481]	valid_0's multi_logloss: 0.156513
[482]	valid_0's multi_logloss: 0.156495
[483]	valid_0's multi_logloss: 0.156478
[484]	valid_0's multi_logloss: 0.156465
[485]	valid_0's multi_logloss: 0.156453
[486]	valid_0's multi_logloss: 0.156432
[487]	valid_0's multi_logloss: 0.156413
[488]	valid_0's multi_logloss: 0.156396
[489]	valid_0's multi_logloss: 0.156383


[673]	valid_0's multi_logloss: 0.155177
[674]	valid_0's multi_logloss: 0.155167
[675]	valid_0's multi_logloss: 0.155163
[676]	valid_0's multi_logloss: 0.155155
[677]	valid_0's multi_logloss: 0.155148
[678]	valid_0's multi_logloss: 0.155144
[679]	valid_0's multi_logloss: 0.155145
[680]	valid_0's multi_logloss: 0.155145
[681]	valid_0's multi_logloss: 0.155143
[682]	valid_0's multi_logloss: 0.155137
[683]	valid_0's multi_logloss: 0.155141
[684]	valid_0's multi_logloss: 0.155143
[685]	valid_0's multi_logloss: 0.155145
[686]	valid_0's multi_logloss: 0.155143
[687]	valid_0's multi_logloss: 0.155149
[688]	valid_0's multi_logloss: 0.155143
[689]	valid_0's multi_logloss: 0.155145
[690]	valid_0's multi_logloss: 0.155137
[691]	valid_0's multi_logloss: 0.15513
[692]	valid_0's multi_logloss: 0.155126
[693]	valid_0's multi_logloss: 0.155125
[694]	valid_0's multi_logloss: 0.155123
[695]	valid_0's multi_logloss: 0.155137
[696]	valid_0's multi_logloss: 0.155139
[697]	valid_0's multi_logloss: 0.155134
[

[879]	valid_0's multi_logloss: 0.155186
[880]	valid_0's multi_logloss: 0.155191
[881]	valid_0's multi_logloss: 0.155199
[882]	valid_0's multi_logloss: 0.155203
[883]	valid_0's multi_logloss: 0.155209
[884]	valid_0's multi_logloss: 0.155208
[885]	valid_0's multi_logloss: 0.155216
[886]	valid_0's multi_logloss: 0.155214
[887]	valid_0's multi_logloss: 0.155217
[888]	valid_0's multi_logloss: 0.155215
[889]	valid_0's multi_logloss: 0.155213
[890]	valid_0's multi_logloss: 0.15522
[891]	valid_0's multi_logloss: 0.155216
[892]	valid_0's multi_logloss: 0.155224
[893]	valid_0's multi_logloss: 0.155223
[894]	valid_0's multi_logloss: 0.155242
[895]	valid_0's multi_logloss: 0.155245
[896]	valid_0's multi_logloss: 0.155242
[897]	valid_0's multi_logloss: 0.155248
[898]	valid_0's multi_logloss: 0.155257
[899]	valid_0's multi_logloss: 0.155262
[900]	valid_0's multi_logloss: 0.155252
[901]	valid_0's multi_logloss: 0.155255
Early stopping, best iteration is:
[801]	valid_0's multi_logloss: 0.15506


In [15]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.5594%


In [16]:
print(p_val.shape, p_tst.shape)

(320000, 3) (80000, 3)


In [17]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

## 제출 파일 생성

In [18]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [19]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [20]:
sub[target_col].value_counts()

2    40733
0    29981
1     9286
Name: class, dtype: int64

In [21]:
sub.to_csv(sub_file)

# 두번째 Hyperparameter Tuning

In [22]:
algo_name = 'lgb_hyperopt_last2'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = data_dir / f'{feature_name}.csv'
p_val_file = data_dir / f'{model_name}.val.csv'
p_tst_file = data_dir / f'{model_name}.tst.csv'
sub_file = data_dir / f'{model_name}.csv'

In [23]:
params = {
    "objective": "multiclass",
    "n_estimators": 10000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [127, 160, 255]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1),
    "min_child_samples": hp.choice('min_child_samples', [100, 130, 173, 200, 255]),
    "max_depth" : hp.choice("max_depth", [30, 40, 50, 60])
}

In [24]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=100,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [05:21<00:00, 32.14s/trial, best loss: 0.15717643888258512]
{'objective': 'multiclass', 'n_estimators': 10000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.013079773714445665, 'max_depth': 50, 'min_child_samples': 100, 'num_leaves': 127, 'subsample': 0.7000000000000001}


# 두번째 LGBM 모델 학습

In [25]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=100)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.96742
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.949368
[3]	valid_0's multi_logloss: 0.93132
[4]	valid_0's multi_logloss: 0.914354
[5]	valid_0's multi_logloss: 0.898528
[6]	valid_0's multi_logloss: 0.882724
[7]	valid_0's multi_logloss: 0.868256
[8]	valid_0's multi_logloss: 0.852913
[9]	valid_0's multi_logloss: 0.837623
[10]	valid_0's multi_logloss: 0.823379
[11]	valid_0's multi_logloss: 0.810312
[12]	valid_0's multi_logloss: 0.797574
[13]	valid_0's multi_logloss: 0.784651
[14]	valid_0's multi_logloss: 0.771972
[15]	valid_0's multi_logloss: 0.760084
[16]	valid_0's multi_logloss: 0.748377
[17]	valid_0's multi_logloss: 0.73667
[18]	valid_0's multi_logloss: 0.724512
[19]	valid_0's multi_logloss: 0.713011
[20]	valid_0's multi_logloss: 0.701832
[21]	valid_0's multi_logloss: 0.69071
[22]	valid_0's multi_logloss: 0.679729
[23]	valid_0's multi_logloss: 0.669135
[24]	valid_0's multi_logloss:

[207]	valid_0's multi_logloss: 0.19054
[208]	valid_0's multi_logloss: 0.190218
[209]	valid_0's multi_logloss: 0.189839
[210]	valid_0's multi_logloss: 0.189393
[211]	valid_0's multi_logloss: 0.189098
[212]	valid_0's multi_logloss: 0.188835
[213]	valid_0's multi_logloss: 0.188496
[214]	valid_0's multi_logloss: 0.188136
[215]	valid_0's multi_logloss: 0.187741
[216]	valid_0's multi_logloss: 0.187297
[217]	valid_0's multi_logloss: 0.186864
[218]	valid_0's multi_logloss: 0.186638
[219]	valid_0's multi_logloss: 0.186193
[220]	valid_0's multi_logloss: 0.185797
[221]	valid_0's multi_logloss: 0.185562
[222]	valid_0's multi_logloss: 0.1852
[223]	valid_0's multi_logloss: 0.184861
[224]	valid_0's multi_logloss: 0.18457
[225]	valid_0's multi_logloss: 0.184215
[226]	valid_0's multi_logloss: 0.183856
[227]	valid_0's multi_logloss: 0.183578
[228]	valid_0's multi_logloss: 0.183302
[229]	valid_0's multi_logloss: 0.18301
[230]	valid_0's multi_logloss: 0.182613
[231]	valid_0's multi_logloss: 0.1823
[232]	v

[413]	valid_0's multi_logloss: 0.158815
[414]	valid_0's multi_logloss: 0.158775
[415]	valid_0's multi_logloss: 0.158731
[416]	valid_0's multi_logloss: 0.15869
[417]	valid_0's multi_logloss: 0.15863
[418]	valid_0's multi_logloss: 0.158586
[419]	valid_0's multi_logloss: 0.158567
[420]	valid_0's multi_logloss: 0.158512
[421]	valid_0's multi_logloss: 0.158467
[422]	valid_0's multi_logloss: 0.158411
[423]	valid_0's multi_logloss: 0.158366
[424]	valid_0's multi_logloss: 0.158323
[425]	valid_0's multi_logloss: 0.158278
[426]	valid_0's multi_logloss: 0.158241
[427]	valid_0's multi_logloss: 0.15821
[428]	valid_0's multi_logloss: 0.158169
[429]	valid_0's multi_logloss: 0.15811
[430]	valid_0's multi_logloss: 0.158081
[431]	valid_0's multi_logloss: 0.158035
[432]	valid_0's multi_logloss: 0.158002
[433]	valid_0's multi_logloss: 0.15797
[434]	valid_0's multi_logloss: 0.157954
[435]	valid_0's multi_logloss: 0.157909
[436]	valid_0's multi_logloss: 0.157866
[437]	valid_0's multi_logloss: 0.157836
[438]

[619]	valid_0's multi_logloss: 0.154088
[620]	valid_0's multi_logloss: 0.154081
[621]	valid_0's multi_logloss: 0.154069
[622]	valid_0's multi_logloss: 0.15406
[623]	valid_0's multi_logloss: 0.154044
[624]	valid_0's multi_logloss: 0.154036
[625]	valid_0's multi_logloss: 0.154029
[626]	valid_0's multi_logloss: 0.154014
[627]	valid_0's multi_logloss: 0.154004
[628]	valid_0's multi_logloss: 0.153998
[629]	valid_0's multi_logloss: 0.153986
[630]	valid_0's multi_logloss: 0.153975
[631]	valid_0's multi_logloss: 0.153963
[632]	valid_0's multi_logloss: 0.153958
[633]	valid_0's multi_logloss: 0.153949
[634]	valid_0's multi_logloss: 0.153936
[635]	valid_0's multi_logloss: 0.153932
[636]	valid_0's multi_logloss: 0.153927
[637]	valid_0's multi_logloss: 0.153913
[638]	valid_0's multi_logloss: 0.153904
[639]	valid_0's multi_logloss: 0.153904
[640]	valid_0's multi_logloss: 0.153895
[641]	valid_0's multi_logloss: 0.153888
[642]	valid_0's multi_logloss: 0.153875
[643]	valid_0's multi_logloss: 0.153862
[

[827]	valid_0's multi_logloss: 0.152916
[828]	valid_0's multi_logloss: 0.152926
[829]	valid_0's multi_logloss: 0.152919
[830]	valid_0's multi_logloss: 0.152917
[831]	valid_0's multi_logloss: 0.152914
[832]	valid_0's multi_logloss: 0.152909
[833]	valid_0's multi_logloss: 0.152911
[834]	valid_0's multi_logloss: 0.152913
[835]	valid_0's multi_logloss: 0.152911
[836]	valid_0's multi_logloss: 0.152907
[837]	valid_0's multi_logloss: 0.152905
[838]	valid_0's multi_logloss: 0.152905
[839]	valid_0's multi_logloss: 0.152899
[840]	valid_0's multi_logloss: 0.152898
[841]	valid_0's multi_logloss: 0.152897
[842]	valid_0's multi_logloss: 0.152899
[843]	valid_0's multi_logloss: 0.152902
[844]	valid_0's multi_logloss: 0.152898
[845]	valid_0's multi_logloss: 0.15289
[846]	valid_0's multi_logloss: 0.15289
[847]	valid_0's multi_logloss: 0.152886
[848]	valid_0's multi_logloss: 0.152879
[849]	valid_0's multi_logloss: 0.152872
[850]	valid_0's multi_logloss: 0.152872
[851]	valid_0's multi_logloss: 0.152861
[8

...

[192]	valid_0's multi_logloss: 0.199269
[193]	valid_0's multi_logloss: 0.198791
[194]	valid_0's multi_logloss: 0.198344
[195]	valid_0's multi_logloss: 0.197903
[196]	valid_0's multi_logloss: 0.197458
[197]	valid_0's multi_logloss: 0.196885
[198]	valid_0's multi_logloss: 0.1965
[199]	valid_0's multi_logloss: 0.195969
[200]	valid_0's multi_logloss: 0.195507
[201]	valid_0's multi_logloss: 0.195049
[202]	valid_0's multi_logloss: 0.194544
[203]	valid_0's multi_logloss: 0.194162
[204]	valid_0's multi_logloss: 0.193785
[205]	valid_0's multi_logloss: 0.193299
[206]	valid_0's multi_logloss: 0.192858
[207]	valid_0's multi_logloss: 0.192365
[208]	valid_0's multi_logloss: 0.192035
[209]	valid_0's multi_logloss: 0.191652
[210]	valid_0's multi_logloss: 0.1912
[211]	valid_0's multi_logloss: 0.190915
[212]	valid_0's multi_logloss: 0.19064
[213]	valid_0's multi_logloss: 0.190291
[214]	valid_0's multi_logloss: 0.189929
[215]	valid_0's multi_logloss: 0.189528
[216]	valid_0's multi_logloss: 0.189074
[217]

[398]	valid_0's multi_logloss: 0.161205
[399]	valid_0's multi_logloss: 0.161144
[400]	valid_0's multi_logloss: 0.161085
[401]	valid_0's multi_logloss: 0.161036
[402]	valid_0's multi_logloss: 0.161
[403]	valid_0's multi_logloss: 0.160961
[404]	valid_0's multi_logloss: 0.160916
[405]	valid_0's multi_logloss: 0.160852
[406]	valid_0's multi_logloss: 0.160786
[407]	valid_0's multi_logloss: 0.160738
[408]	valid_0's multi_logloss: 0.160678
[409]	valid_0's multi_logloss: 0.160646
[410]	valid_0's multi_logloss: 0.1606
[411]	valid_0's multi_logloss: 0.160548
[412]	valid_0's multi_logloss: 0.160494
[413]	valid_0's multi_logloss: 0.160427
[414]	valid_0's multi_logloss: 0.160383
[415]	valid_0's multi_logloss: 0.160333
[416]	valid_0's multi_logloss: 0.16029
[417]	valid_0's multi_logloss: 0.160236
[418]	valid_0's multi_logloss: 0.160164
[419]	valid_0's multi_logloss: 0.160143
[420]	valid_0's multi_logloss: 0.160081
[421]	valid_0's multi_logloss: 0.160039
[422]	valid_0's multi_logloss: 0.159986
[423]	

[607]	valid_0's multi_logloss: 0.156046
[608]	valid_0's multi_logloss: 0.156034
[609]	valid_0's multi_logloss: 0.156022
[610]	valid_0's multi_logloss: 0.156012
[611]	valid_0's multi_logloss: 0.156001
[612]	valid_0's multi_logloss: 0.155996
[613]	valid_0's multi_logloss: 0.155985
[614]	valid_0's multi_logloss: 0.155986
[615]	valid_0's multi_logloss: 0.155976
[616]	valid_0's multi_logloss: 0.155971
[617]	valid_0's multi_logloss: 0.155962
[618]	valid_0's multi_logloss: 0.155953
[619]	valid_0's multi_logloss: 0.155945
[620]	valid_0's multi_logloss: 0.155936
[621]	valid_0's multi_logloss: 0.155918
[622]	valid_0's multi_logloss: 0.155914
[623]	valid_0's multi_logloss: 0.155906
[624]	valid_0's multi_logloss: 0.155891
[625]	valid_0's multi_logloss: 0.155885
[626]	valid_0's multi_logloss: 0.155871
[627]	valid_0's multi_logloss: 0.155865
[628]	valid_0's multi_logloss: 0.155858
[629]	valid_0's multi_logloss: 0.155842
[630]	valid_0's multi_logloss: 0.155836
[631]	valid_0's multi_logloss: 0.155821


[814]	valid_0's multi_logloss: 0.155012
[815]	valid_0's multi_logloss: 0.155016
[816]	valid_0's multi_logloss: 0.155018
[817]	valid_0's multi_logloss: 0.155018
[818]	valid_0's multi_logloss: 0.155014
[819]	valid_0's multi_logloss: 0.155021
[820]	valid_0's multi_logloss: 0.15502
[821]	valid_0's multi_logloss: 0.155008
[822]	valid_0's multi_logloss: 0.155003
[823]	valid_0's multi_logloss: 0.154994
[824]	valid_0's multi_logloss: 0.154986
[825]	valid_0's multi_logloss: 0.154977
[826]	valid_0's multi_logloss: 0.154977
[827]	valid_0's multi_logloss: 0.154966
[828]	valid_0's multi_logloss: 0.154967
[829]	valid_0's multi_logloss: 0.154966
[830]	valid_0's multi_logloss: 0.154962
[831]	valid_0's multi_logloss: 0.154967
[832]	valid_0's multi_logloss: 0.154965
[833]	valid_0's multi_logloss: 0.154965
[834]	valid_0's multi_logloss: 0.154964
[835]	valid_0's multi_logloss: 0.154966
[836]	valid_0's multi_logloss: 0.154973
[837]	valid_0's multi_logloss: 0.154966
[838]	valid_0's multi_logloss: 0.154963
[

[1021]	valid_0's multi_logloss: 0.154812
[1022]	valid_0's multi_logloss: 0.154819
[1023]	valid_0's multi_logloss: 0.154828
[1024]	valid_0's multi_logloss: 0.154831
[1025]	valid_0's multi_logloss: 0.15483
[1026]	valid_0's multi_logloss: 0.154834
[1027]	valid_0's multi_logloss: 0.154833
[1028]	valid_0's multi_logloss: 0.15483
[1029]	valid_0's multi_logloss: 0.154827
[1030]	valid_0's multi_logloss: 0.154825
[1031]	valid_0's multi_logloss: 0.154828
[1032]	valid_0's multi_logloss: 0.154831
[1033]	valid_0's multi_logloss: 0.154834
[1034]	valid_0's multi_logloss: 0.154831
[1035]	valid_0's multi_logloss: 0.154827
[1036]	valid_0's multi_logloss: 0.154827
[1037]	valid_0's multi_logloss: 0.154826
[1038]	valid_0's multi_logloss: 0.154826
[1039]	valid_0's multi_logloss: 0.154829
[1040]	valid_0's multi_logloss: 0.154827
[1041]	valid_0's multi_logloss: 0.15483
[1042]	valid_0's multi_logloss: 0.154834
[1043]	valid_0's multi_logloss: 0.154837
[1044]	valid_0's multi_logloss: 0.154837
[1045]	valid_0's mu

In [26]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.5478%


In [27]:
print(p_val.shape, p_tst.shape)

(320000, 3) (80000, 3)


In [28]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [29]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [30]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [31]:
sub[target_col].value_counts()

2    40702
0    29974
1     9324
Name: class, dtype: int64

In [32]:
sub.to_csv(sub_file)

# 세번째 Hyperparameter Tuning

In [33]:
algo_name = 'lgb_hyperopt_last3'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = data_dir / f'{feature_name}.csv'
p_val_file = data_dir / f'{model_name}.val.csv'
p_tst_file = data_dir / f'{model_name}.tst.csv'
sub_file = data_dir / f'{model_name}.csv'

In [34]:
params = {
    "objective": "multiclass",
    "n_estimators": 10000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [100, 127, 135, 160]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1),
    "min_child_samples": hp.choice('min_child_samples', [50, 75, 100, 130, 173]),
    "max_depth" : hp.choice("max_depth", [40, 45, 50, 55, 60])
}

In [35]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=100,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:26<00:00, 14.65s/trial, best loss: 0.1574318010952755]
{'objective': 'multiclass', 'n_estimators': 10000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'colsample_bytree': 0.9, 'learning_rate': 0.014570543076307067, 'max_depth': 60, 'min_child_samples': 50, 'num_leaves': 160, 'subsample': 0.7000000000000001}


# 세번째 LGBM 모델 학습

In [36]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=100)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.963527
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.942004
[3]	valid_0's multi_logloss: 0.921311
[4]	valid_0's multi_logloss: 0.902394
[5]	valid_0's multi_logloss: 0.883239
[6]	valid_0's multi_logloss: 0.864752
[7]	valid_0's multi_logloss: 0.848384
[8]	valid_0's multi_logloss: 0.831137
[9]	valid_0's multi_logloss: 0.814469
[10]	valid_0's multi_logloss: 0.79836
[11]	valid_0's multi_logloss: 0.783541
[12]	valid_0's multi_logloss: 0.768427
[13]	valid_0's multi_logloss: 0.754551
[14]	valid_0's multi_logloss: 0.74034
[15]	valid_0's multi_logloss: 0.726544
[16]	valid_0's multi_logloss: 0.71321
[17]	valid_0's multi_logloss: 0.70025
[18]	valid_0's multi_logloss: 0.687683
[19]	valid_0's multi_logloss: 0.675564
[20]	valid_0's multi_logloss: 0.663785
[21]	valid_0's multi_logloss: 0.652296
[22]	valid_0's multi_logloss: 0.641173
[23]	valid_0's multi_logloss: 0.630316
[24]	valid_0's multi_logloss:

[207]	valid_0's multi_logloss: 0.173807
[208]	valid_0's multi_logloss: 0.173524
[209]	valid_0's multi_logloss: 0.173244
[210]	valid_0's multi_logloss: 0.172975
[211]	valid_0's multi_logloss: 0.17271
[212]	valid_0's multi_logloss: 0.172461
[213]	valid_0's multi_logloss: 0.172186
[214]	valid_0's multi_logloss: 0.171933
[215]	valid_0's multi_logloss: 0.17169
[216]	valid_0's multi_logloss: 0.171438
[217]	valid_0's multi_logloss: 0.171194
[218]	valid_0's multi_logloss: 0.170976
[219]	valid_0's multi_logloss: 0.170731
[220]	valid_0's multi_logloss: 0.170504
[221]	valid_0's multi_logloss: 0.170313
[222]	valid_0's multi_logloss: 0.170096
[223]	valid_0's multi_logloss: 0.169878
[224]	valid_0's multi_logloss: 0.169651
[225]	valid_0's multi_logloss: 0.16944
[226]	valid_0's multi_logloss: 0.169216
[227]	valid_0's multi_logloss: 0.169007
[228]	valid_0's multi_logloss: 0.168798
[229]	valid_0's multi_logloss: 0.168594
[230]	valid_0's multi_logloss: 0.168396
[231]	valid_0's multi_logloss: 0.168194
[23

[412]	valid_0's multi_logloss: 0.154929
[413]	valid_0's multi_logloss: 0.154905
[414]	valid_0's multi_logloss: 0.154881
[415]	valid_0's multi_logloss: 0.15485
[416]	valid_0's multi_logloss: 0.15483
[417]	valid_0's multi_logloss: 0.154812
[418]	valid_0's multi_logloss: 0.154797
[419]	valid_0's multi_logloss: 0.154787
[420]	valid_0's multi_logloss: 0.154774
[421]	valid_0's multi_logloss: 0.154758
[422]	valid_0's multi_logloss: 0.15474
[423]	valid_0's multi_logloss: 0.154722
[424]	valid_0's multi_logloss: 0.15471
[425]	valid_0's multi_logloss: 0.154692
[426]	valid_0's multi_logloss: 0.154671
[427]	valid_0's multi_logloss: 0.154649
[428]	valid_0's multi_logloss: 0.154636
[429]	valid_0's multi_logloss: 0.154616
[430]	valid_0's multi_logloss: 0.154596
[431]	valid_0's multi_logloss: 0.154579
[432]	valid_0's multi_logloss: 0.154567
[433]	valid_0's multi_logloss: 0.154545
[434]	valid_0's multi_logloss: 0.154529
[435]	valid_0's multi_logloss: 0.154516
[436]	valid_0's multi_logloss: 0.154499
[437

[617]	valid_0's multi_logloss: 0.15291
[618]	valid_0's multi_logloss: 0.152914
[619]	valid_0's multi_logloss: 0.152902
[620]	valid_0's multi_logloss: 0.1529
[621]	valid_0's multi_logloss: 0.152905
[622]	valid_0's multi_logloss: 0.152904
[623]	valid_0's multi_logloss: 0.152901
[624]	valid_0's multi_logloss: 0.152897
[625]	valid_0's multi_logloss: 0.152885
[626]	valid_0's multi_logloss: 0.152877
[627]	valid_0's multi_logloss: 0.15287
[628]	valid_0's multi_logloss: 0.152868
[629]	valid_0's multi_logloss: 0.152875
[630]	valid_0's multi_logloss: 0.152882
[631]	valid_0's multi_logloss: 0.15288
[632]	valid_0's multi_logloss: 0.152889
[633]	valid_0's multi_logloss: 0.152872
[634]	valid_0's multi_logloss: 0.15287
[635]	valid_0's multi_logloss: 0.152869
[636]	valid_0's multi_logloss: 0.152867
[637]	valid_0's multi_logloss: 0.152865
[638]	valid_0's multi_logloss: 0.152861
[639]	valid_0's multi_logloss: 0.152865
[640]	valid_0's multi_logloss: 0.152864
[641]	valid_0's multi_logloss: 0.152868
[642]	

[823]	valid_0's multi_logloss: 0.152809
[824]	valid_0's multi_logloss: 0.152814
[825]	valid_0's multi_logloss: 0.152826
[826]	valid_0's multi_logloss: 0.152827
[827]	valid_0's multi_logloss: 0.15283
[828]	valid_0's multi_logloss: 0.152824
[829]	valid_0's multi_logloss: 0.152829
[830]	valid_0's multi_logloss: 0.152824
[831]	valid_0's multi_logloss: 0.152823
[832]	valid_0's multi_logloss: 0.152821
[833]	valid_0's multi_logloss: 0.152822
[834]	valid_0's multi_logloss: 0.152831
[835]	valid_0's multi_logloss: 0.152835
[836]	valid_0's multi_logloss: 0.152843
[837]	valid_0's multi_logloss: 0.152852
[838]	valid_0's multi_logloss: 0.152868
[839]	valid_0's multi_logloss: 0.152858
[840]	valid_0's multi_logloss: 0.152861
[841]	valid_0's multi_logloss: 0.15286
[842]	valid_0's multi_logloss: 0.152863
[843]	valid_0's multi_logloss: 0.152868
[844]	valid_0's multi_logloss: 0.152873
[845]	valid_0's multi_logloss: 0.152877
[846]	valid_0's multi_logloss: 0.15288
[847]	valid_0's multi_logloss: 0.152884
[84

...

[802]	valid_0's multi_logloss: 0.150774
[803]	valid_0's multi_logloss: 0.150772
[804]	valid_0's multi_logloss: 0.150776
[805]	valid_0's multi_logloss: 0.150774
[806]	valid_0's multi_logloss: 0.15077
[807]	valid_0's multi_logloss: 0.150772
[808]	valid_0's multi_logloss: 0.150771
[809]	valid_0's multi_logloss: 0.15077
[810]	valid_0's multi_logloss: 0.150785
[811]	valid_0's multi_logloss: 0.150784
[812]	valid_0's multi_logloss: 0.150787
[813]	valid_0's multi_logloss: 0.150795
[814]	valid_0's multi_logloss: 0.150798
[815]	valid_0's multi_logloss: 0.150798
[816]	valid_0's multi_logloss: 0.150799
[817]	valid_0's multi_logloss: 0.150802
[818]	valid_0's multi_logloss: 0.150805
[819]	valid_0's multi_logloss: 0.150811
[820]	valid_0's multi_logloss: 0.150814
[821]	valid_0's multi_logloss: 0.150815
[822]	valid_0's multi_logloss: 0.150816
[823]	valid_0's multi_logloss: 0.150813
[824]	valid_0's multi_logloss: 0.150813
[825]	valid_0's multi_logloss: 0.150812
[826]	valid_0's multi_logloss: 0.150807
[8

[129]	valid_0's multi_logloss: 0.221314
[130]	valid_0's multi_logloss: 0.220302
[131]	valid_0's multi_logloss: 0.219242
[132]	valid_0's multi_logloss: 0.218214
[133]	valid_0's multi_logloss: 0.217274
[134]	valid_0's multi_logloss: 0.216284
[135]	valid_0's multi_logloss: 0.2153
[136]	valid_0's multi_logloss: 0.214347
[137]	valid_0's multi_logloss: 0.213404
[138]	valid_0's multi_logloss: 0.212472
[139]	valid_0's multi_logloss: 0.211553
[140]	valid_0's multi_logloss: 0.210663
[141]	valid_0's multi_logloss: 0.209775
[142]	valid_0's multi_logloss: 0.208913
[143]	valid_0's multi_logloss: 0.208064
[144]	valid_0's multi_logloss: 0.207244
[145]	valid_0's multi_logloss: 0.206413
[146]	valid_0's multi_logloss: 0.205632
[147]	valid_0's multi_logloss: 0.204837
[148]	valid_0's multi_logloss: 0.204059
[149]	valid_0's multi_logloss: 0.2033
[150]	valid_0's multi_logloss: 0.202545
[151]	valid_0's multi_logloss: 0.201813
[152]	valid_0's multi_logloss: 0.201095
[153]	valid_0's multi_logloss: 0.200391
[154

[334]	valid_0's multi_logloss: 0.159182
[335]	valid_0's multi_logloss: 0.159139
[336]	valid_0's multi_logloss: 0.159089
[337]	valid_0's multi_logloss: 0.159025
[338]	valid_0's multi_logloss: 0.15898
[339]	valid_0's multi_logloss: 0.158937
[340]	valid_0's multi_logloss: 0.158894
[341]	valid_0's multi_logloss: 0.158855
[342]	valid_0's multi_logloss: 0.15881
[343]	valid_0's multi_logloss: 0.158774
[344]	valid_0's multi_logloss: 0.158731
[345]	valid_0's multi_logloss: 0.158694
[346]	valid_0's multi_logloss: 0.158646
[347]	valid_0's multi_logloss: 0.158595
[348]	valid_0's multi_logloss: 0.158547
[349]	valid_0's multi_logloss: 0.158514
[350]	valid_0's multi_logloss: 0.158476
[351]	valid_0's multi_logloss: 0.158436
[352]	valid_0's multi_logloss: 0.158404
[353]	valid_0's multi_logloss: 0.158366
[354]	valid_0's multi_logloss: 0.15832
[355]	valid_0's multi_logloss: 0.158285
[356]	valid_0's multi_logloss: 0.158245
[357]	valid_0's multi_logloss: 0.158198
[358]	valid_0's multi_logloss: 0.158165
[35

[542]	valid_0's multi_logloss: 0.155091
[543]	valid_0's multi_logloss: 0.155083
[544]	valid_0's multi_logloss: 0.155078
[545]	valid_0's multi_logloss: 0.155072
[546]	valid_0's multi_logloss: 0.155065
[547]	valid_0's multi_logloss: 0.155065
[548]	valid_0's multi_logloss: 0.155063
[549]	valid_0's multi_logloss: 0.155059
[550]	valid_0's multi_logloss: 0.155053
[551]	valid_0's multi_logloss: 0.155044
[552]	valid_0's multi_logloss: 0.15504
[553]	valid_0's multi_logloss: 0.15504
[554]	valid_0's multi_logloss: 0.155033
[555]	valid_0's multi_logloss: 0.155026
[556]	valid_0's multi_logloss: 0.155015
[557]	valid_0's multi_logloss: 0.155011
[558]	valid_0's multi_logloss: 0.155001
[559]	valid_0's multi_logloss: 0.155003
[560]	valid_0's multi_logloss: 0.154988
[561]	valid_0's multi_logloss: 0.154979
[562]	valid_0's multi_logloss: 0.154977
[563]	valid_0's multi_logloss: 0.154973
[564]	valid_0's multi_logloss: 0.154968
[565]	valid_0's multi_logloss: 0.154972
[566]	valid_0's multi_logloss: 0.154972
[5

[747]	valid_0's multi_logloss: 0.154609
[748]	valid_0's multi_logloss: 0.154602
[749]	valid_0's multi_logloss: 0.154598
[750]	valid_0's multi_logloss: 0.154608
[751]	valid_0's multi_logloss: 0.154605
[752]	valid_0's multi_logloss: 0.154599
[753]	valid_0's multi_logloss: 0.154588
[754]	valid_0's multi_logloss: 0.154592
[755]	valid_0's multi_logloss: 0.154591
[756]	valid_0's multi_logloss: 0.154585
[757]	valid_0's multi_logloss: 0.154596
[758]	valid_0's multi_logloss: 0.154595
[759]	valid_0's multi_logloss: 0.154601
[760]	valid_0's multi_logloss: 0.154607
[761]	valid_0's multi_logloss: 0.154608
[762]	valid_0's multi_logloss: 0.154611
[763]	valid_0's multi_logloss: 0.154616
[764]	valid_0's multi_logloss: 0.154621
[765]	valid_0's multi_logloss: 0.154619
[766]	valid_0's multi_logloss: 0.154636
[767]	valid_0's multi_logloss: 0.154638
[768]	valid_0's multi_logloss: 0.154644
[769]	valid_0's multi_logloss: 0.154646
[770]	valid_0's multi_logloss: 0.154655
[771]	valid_0's multi_logloss: 0.154656


In [37]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.5566%


In [38]:
print(p_val.shape, p_tst.shape)

(320000, 3) (80000, 3)


In [39]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [40]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [41]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [42]:
sub[target_col].value_counts()

2    40683
0    29976
1     9341
Name: class, dtype: int64

In [43]:
sub.to_csv(sub_file)

# 네번째 Hyperparameter Tuning

In [44]:
algo_name = 'lgb_hyperopt_last4'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = data_dir / f'{feature_name}.csv'
p_val_file = data_dir / f'{model_name}.val.csv'
p_tst_file = data_dir / f'{model_name}.tst.csv'
sub_file = data_dir / f'{model_name}.csv'

In [45]:
params = {
    "objective": "multiclass",
    "n_estimators": 10000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [135, 160, 193, 222]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1),
    "min_child_samples": hp.choice('min_child_samples', [30, 50, 75, 100, 130]),
    "max_depth" : hp.choice("max_depth", [50, 55, 60, 65, 70])
}

In [46]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=100,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
3 print(params)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:55<00:00, 23.55s/trial, best loss: 0.15748908676699044]
{'objective': 'multiclass', 'n_estimators': 10000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'colsample_bytree': 0.8, 'learning_rate': 0.018430677003419285, 'max_depth': 60, 'min_child_samples': 50, 'num_leaves': 135, 'subsample': 0.6000000000000001}


# 네번째 LGBM 모델 학습

In [47]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=100)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.95809
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.932706
[3]	valid_0's multi_logloss: 0.907167
[4]	valid_0's multi_logloss: 0.884186
[5]	valid_0's multi_logloss: 0.862165
[6]	valid_0's multi_logloss: 0.841209
[7]	valid_0's multi_logloss: 0.821776
[8]	valid_0's multi_logloss: 0.801959
[9]	valid_0's multi_logloss: 0.782174
[10]	valid_0's multi_logloss: 0.763965
[11]	valid_0's multi_logloss: 0.74709
[12]	valid_0's multi_logloss: 0.72981
[13]	valid_0's multi_logloss: 0.713906
[14]	valid_0's multi_logloss: 0.697889
[15]	valid_0's multi_logloss: 0.682451
[16]	valid_0's multi_logloss: 0.668067
[17]	valid_0's multi_logloss: 0.65344
[18]	valid_0's multi_logloss: 0.639339
[19]	valid_0's multi_logloss: 0.625872
[20]	valid_0's multi_logloss: 0.612795
[21]	valid_0's multi_logloss: 0.600221
[22]	valid_0's multi_logloss: 0.587983
[23]	valid_0's multi_logloss: 0.576165
[24]	valid_0's multi_logloss:

[206]	valid_0's multi_logloss: 0.165149
[207]	valid_0's multi_logloss: 0.164958
[208]	valid_0's multi_logloss: 0.164763
[209]	valid_0's multi_logloss: 0.164594
[210]	valid_0's multi_logloss: 0.164418
[211]	valid_0's multi_logloss: 0.164275
[212]	valid_0's multi_logloss: 0.16413
[213]	valid_0's multi_logloss: 0.163969
[214]	valid_0's multi_logloss: 0.16382
[215]	valid_0's multi_logloss: 0.163664
[216]	valid_0's multi_logloss: 0.163518
[217]	valid_0's multi_logloss: 0.163357
[218]	valid_0's multi_logloss: 0.163239
[219]	valid_0's multi_logloss: 0.163083
[220]	valid_0's multi_logloss: 0.162956
[221]	valid_0's multi_logloss: 0.162846
[222]	valid_0's multi_logloss: 0.162705
[223]	valid_0's multi_logloss: 0.162575
[224]	valid_0's multi_logloss: 0.162461
[225]	valid_0's multi_logloss: 0.162317
[226]	valid_0's multi_logloss: 0.162204
[227]	valid_0's multi_logloss: 0.162076
[228]	valid_0's multi_logloss: 0.16197
[229]	valid_0's multi_logloss: 0.161829
[230]	valid_0's multi_logloss: 0.161696
[23

[414]	valid_0's multi_logloss: 0.153806
[415]	valid_0's multi_logloss: 0.153806
[416]	valid_0's multi_logloss: 0.153789
[417]	valid_0's multi_logloss: 0.153766
[418]	valid_0's multi_logloss: 0.153769
[419]	valid_0's multi_logloss: 0.153751
[420]	valid_0's multi_logloss: 0.153745
[421]	valid_0's multi_logloss: 0.153728
[422]	valid_0's multi_logloss: 0.153716
[423]	valid_0's multi_logloss: 0.153702
[424]	valid_0's multi_logloss: 0.153686
[425]	valid_0's multi_logloss: 0.153662
[426]	valid_0's multi_logloss: 0.153658
[427]	valid_0's multi_logloss: 0.15365
[428]	valid_0's multi_logloss: 0.153636
[429]	valid_0's multi_logloss: 0.153626
[430]	valid_0's multi_logloss: 0.153626
[431]	valid_0's multi_logloss: 0.153609
[432]	valid_0's multi_logloss: 0.153601
[433]	valid_0's multi_logloss: 0.153596
[434]	valid_0's multi_logloss: 0.153565
[435]	valid_0's multi_logloss: 0.153557
[436]	valid_0's multi_logloss: 0.153557
[437]	valid_0's multi_logloss: 0.153555
[438]	valid_0's multi_logloss: 0.153558
[

[619]	valid_0's multi_logloss: 0.153105
[620]	valid_0's multi_logloss: 0.153102
[621]	valid_0's multi_logloss: 0.153107
[622]	valid_0's multi_logloss: 0.153105
[623]	valid_0's multi_logloss: 0.153115
[624]	valid_0's multi_logloss: 0.153116
[625]	valid_0's multi_logloss: 0.153104
[626]	valid_0's multi_logloss: 0.153098
[627]	valid_0's multi_logloss: 0.153108
[628]	valid_0's multi_logloss: 0.153103
[629]	valid_0's multi_logloss: 0.153085
[630]	valid_0's multi_logloss: 0.153094
[631]	valid_0's multi_logloss: 0.15309
[632]	valid_0's multi_logloss: 0.15309
[633]	valid_0's multi_logloss: 0.153092
[634]	valid_0's multi_logloss: 0.153098
[635]	valid_0's multi_logloss: 0.153094
[636]	valid_0's multi_logloss: 0.153094
[637]	valid_0's multi_logloss: 0.153091
[638]	valid_0's multi_logloss: 0.153093
[639]	valid_0's multi_logloss: 0.153101
[640]	valid_0's multi_logloss: 0.153095
[641]	valid_0's multi_logloss: 0.153084
[642]	valid_0's multi_logloss: 0.153088
[643]	valid_0's multi_logloss: 0.153087
[6

[125]	valid_0's multi_logloss: 0.200982
[126]	valid_0's multi_logloss: 0.200071
[127]	valid_0's multi_logloss: 0.199179
[128]	valid_0's multi_logloss: 0.198358
[129]	valid_0's multi_logloss: 0.197609
[130]	valid_0's multi_logloss: 0.196878
[131]	valid_0's multi_logloss: 0.196065
[132]	valid_0's multi_logloss: 0.195301
[133]	valid_0's multi_logloss: 0.194615
[134]	valid_0's multi_logloss: 0.193982
[135]	valid_0's multi_logloss: 0.193246
[136]	valid_0's multi_logloss: 0.192506
[137]	valid_0's multi_logloss: 0.191796
[138]	valid_0's multi_logloss: 0.191087
[139]	valid_0's multi_logloss: 0.190415
[140]	valid_0's multi_logloss: 0.18974
[141]	valid_0's multi_logloss: 0.189098
[142]	valid_0's multi_logloss: 0.188451
[143]	valid_0's multi_logloss: 0.187827
[144]	valid_0's multi_logloss: 0.187238
[145]	valid_0's multi_logloss: 0.186654
[146]	valid_0's multi_logloss: 0.186074
[147]	valid_0's multi_logloss: 0.185478
[148]	valid_0's multi_logloss: 0.184893
[149]	valid_0's multi_logloss: 0.184348
[

...

[580]	valid_0's multi_logloss: 0.150636
[581]	valid_0's multi_logloss: 0.150642
[582]	valid_0's multi_logloss: 0.150644
[583]	valid_0's multi_logloss: 0.150647
[584]	valid_0's multi_logloss: 0.15065
[585]	valid_0's multi_logloss: 0.150658
[586]	valid_0's multi_logloss: 0.150653
[587]	valid_0's multi_logloss: 0.150651
[588]	valid_0's multi_logloss: 0.150646
[589]	valid_0's multi_logloss: 0.150644
[590]	valid_0's multi_logloss: 0.150659
[591]	valid_0's multi_logloss: 0.150659
[592]	valid_0's multi_logloss: 0.150653
[593]	valid_0's multi_logloss: 0.150653
[594]	valid_0's multi_logloss: 0.150658
[595]	valid_0's multi_logloss: 0.150658
[596]	valid_0's multi_logloss: 0.150656
[597]	valid_0's multi_logloss: 0.150656
[598]	valid_0's multi_logloss: 0.150653
[599]	valid_0's multi_logloss: 0.150654
[600]	valid_0's multi_logloss: 0.15065
[601]	valid_0's multi_logloss: 0.150646
[602]	valid_0's multi_logloss: 0.150648
[603]	valid_0's multi_logloss: 0.150653
[604]	valid_0's multi_logloss: 0.15065
[60

[787]	valid_0's multi_logloss: 0.150774
[788]	valid_0's multi_logloss: 0.150776
Early stopping, best iteration is:
[688]	valid_0's multi_logloss: 0.150559
training model for CV #10
[1]	valid_0's multi_logloss: 0.95824
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.933061
[3]	valid_0's multi_logloss: 0.907677
[4]	valid_0's multi_logloss: 0.884913
[5]	valid_0's multi_logloss: 0.862924
[6]	valid_0's multi_logloss: 0.842123
[7]	valid_0's multi_logloss: 0.822802
[8]	valid_0's multi_logloss: 0.803076
[9]	valid_0's multi_logloss: 0.783364
[10]	valid_0's multi_logloss: 0.765221
[11]	valid_0's multi_logloss: 0.748454
[12]	valid_0's multi_logloss: 0.731225
[13]	valid_0's multi_logloss: 0.715373
[14]	valid_0's multi_logloss: 0.699413
[15]	valid_0's multi_logloss: 0.684051
[16]	valid_0's multi_logloss: 0.669752
[17]	valid_0's multi_logloss: 0.655164
[18]	valid_0's multi_logloss: 0.641086
[19]	valid_0's multi_logloss: 0.627635
[20]	valid_0's multi_loglo

[205]	valid_0's multi_logloss: 0.167177
[206]	valid_0's multi_logloss: 0.166964
[207]	valid_0's multi_logloss: 0.166781
[208]	valid_0's multi_logloss: 0.166584
[209]	valid_0's multi_logloss: 0.166394
[210]	valid_0's multi_logloss: 0.166237
[211]	valid_0's multi_logloss: 0.166086
[212]	valid_0's multi_logloss: 0.165927
[213]	valid_0's multi_logloss: 0.165766
[214]	valid_0's multi_logloss: 0.165634
[215]	valid_0's multi_logloss: 0.165492
[216]	valid_0's multi_logloss: 0.16533
[217]	valid_0's multi_logloss: 0.165163
[218]	valid_0's multi_logloss: 0.165034
[219]	valid_0's multi_logloss: 0.164876
[220]	valid_0's multi_logloss: 0.164748
[221]	valid_0's multi_logloss: 0.164645
[222]	valid_0's multi_logloss: 0.164519
[223]	valid_0's multi_logloss: 0.164396
[224]	valid_0's multi_logloss: 0.164261
[225]	valid_0's multi_logloss: 0.16413
[226]	valid_0's multi_logloss: 0.163983
[227]	valid_0's multi_logloss: 0.16385
[228]	valid_0's multi_logloss: 0.163741
[229]	valid_0's multi_logloss: 0.16361
[230

[414]	valid_0's multi_logloss: 0.156043
[415]	valid_0's multi_logloss: 0.156023
[416]	valid_0's multi_logloss: 0.156007
[417]	valid_0's multi_logloss: 0.156002
[418]	valid_0's multi_logloss: 0.155989
[419]	valid_0's multi_logloss: 0.155985
[420]	valid_0's multi_logloss: 0.155978
[421]	valid_0's multi_logloss: 0.155971
[422]	valid_0's multi_logloss: 0.155968
[423]	valid_0's multi_logloss: 0.15597
[424]	valid_0's multi_logloss: 0.155939
[425]	valid_0's multi_logloss: 0.155932
[426]	valid_0's multi_logloss: 0.155929
[427]	valid_0's multi_logloss: 0.155912
[428]	valid_0's multi_logloss: 0.15591
[429]	valid_0's multi_logloss: 0.1559
[430]	valid_0's multi_logloss: 0.155896
[431]	valid_0's multi_logloss: 0.155891
[432]	valid_0's multi_logloss: 0.155884
[433]	valid_0's multi_logloss: 0.155876
[434]	valid_0's multi_logloss: 0.155868
[435]	valid_0's multi_logloss: 0.155883
[436]	valid_0's multi_logloss: 0.155871
[437]	valid_0's multi_logloss: 0.155862
[438]	valid_0's multi_logloss: 0.155855
[439

[620]	valid_0's multi_logloss: 0.155349
[621]	valid_0's multi_logloss: 0.155348
[622]	valid_0's multi_logloss: 0.155353
[623]	valid_0's multi_logloss: 0.155349
[624]	valid_0's multi_logloss: 0.155343
[625]	valid_0's multi_logloss: 0.155342
[626]	valid_0's multi_logloss: 0.155345
[627]	valid_0's multi_logloss: 0.155347
[628]	valid_0's multi_logloss: 0.15535
[629]	valid_0's multi_logloss: 0.155345
[630]	valid_0's multi_logloss: 0.155341
[631]	valid_0's multi_logloss: 0.155342
[632]	valid_0's multi_logloss: 0.155338
[633]	valid_0's multi_logloss: 0.155335
[634]	valid_0's multi_logloss: 0.15532
[635]	valid_0's multi_logloss: 0.155329
[636]	valid_0's multi_logloss: 0.155334
[637]	valid_0's multi_logloss: 0.155327
[638]	valid_0's multi_logloss: 0.155325
[639]	valid_0's multi_logloss: 0.155334
[640]	valid_0's multi_logloss: 0.155331
[641]	valid_0's multi_logloss: 0.155328
[642]	valid_0's multi_logloss: 0.155322
[643]	valid_0's multi_logloss: 0.155321
[644]	valid_0's multi_logloss: 0.155324
[6

In [48]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.5275%


In [49]:
print(p_val.shape, p_tst.shape)

(320000, 3) (80000, 3)


In [50]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [51]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [52]:
sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [53]:
sub[target_col].value_counts()

2    40681
0    29972
1     9347
Name: class, dtype: int64

In [54]:
sub.to_csv(sub_file)